# YouTube

In [5]:
import json, time, schedule, datetime
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
pd.options.display.max_colwidth=50

search_terms_excel=r"C:\Users\Noel\Desktop\Alcohol_marketing\Python\DataCollection_SearchTerms.xlsx"
yt=pd.read_excel(search_terms_excel, sheet_name="youtube")
yt_link_list=yt["link"]
yt

,Most popular YouTube videos based on total global views as of April 2022,channel name,link
0,Baby Shark Dance,Pinkfong Kids' Songs & Stories,https://www.youtube.com/watch?v=XqZsoesa55w
1,Despacito,Luis Fonsi featuring Daddy Yankee,https://www.youtube.com/watch?v=kJQP7kiw5Fk
2,Johny Johny Yes Papa,LooLoo Kids,https://www.youtube.com/watch?v=7GjOOyBoELw
3,Shape of You,Ed Sheeran,https://www.youtube.com/watch?v=JGwWNGJdvx8
4,See You Again,Wiz Khalifa featuring Charlie Puth,https://www.youtube.com/watch?v=RgKAFK5djSk
5,Bath Song,Cocomelon Nursery Rhymes,https://www.youtube.com/watch?v=WRVsOCh907o
6,Learning Colors - Colorful Eggs on a Farm,Miroshka TV,https://www.youtube.com/watch?v=_nAu9D-8srA
7,Phonics Song with Two Words,ChuChu TV Nursery Rhymes & Kids Songs,https://www.youtube.com/watch?v=hq3yfQnllfQ
8,Uptown Funk,Mark Ronson featuring Bruno Mars,https://www.youtube.com/watch?v=OPf0YbXqDm0
9,Masha and the Bear - Recipe for Disaster,Get Movies,https://www.youtube.com/watch?v=KYniUCGPGLs


In [6]:
def youtube_ads_scraper(local_chrome_driver_path, target_video_link, target_video_name, html_dict, number_of_fresh_times):

    # print(f"Start scraping ads for {target_video_name}...")

    chrome_driver=webdriver.Chrome(local_chrome_driver_path)
    chrome_driver.get(target_video_link);  # chrome_driver.maximize_window()
    action = ActionChains(chrome_driver)

    time_list, ad_list, ad_number_list=[],[],[]
    
    for i in range(number_of_fresh_times):
        ad_search_time=datetime.datetime.now().strftime("%H:%M:%S")

        video_pause_find=WebDriverWait(chrome_driver,30).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, html_dict["video_pause"])))
        action.move_to_element(video_pause_find).click().perform()
        
        time.sleep(2)

        try: 
            ad_count_find=chrome_driver.find_element(By.CLASS_NAME, html_dict["ad_count"])
            ad_count_text=str(ad_count_find.text)
            ad_duration_find=chrome_driver.find_element(By.CLASS_NAME, html_dict["ad_duration"])
            ad_duration_text_min=str(ad_duration_find.text.split(":")[0])
            ad_duration_text_second=str(ad_duration_find.text.split(":")[1])
            ad_duration_text=int(ad_duration_text_min)*60 + int(ad_duration_text_second)

            if ad_count_text=="Ad ·":
                action.move_to_element(video_pause_find).click().perform()
                action.move_to_element(video_pause_find).context_click().perform()
                stats_find=chrome_driver.find_element(By.CSS_SELECTOR, html_dict["video_stats"])
                action.move_to_element(stats_find).click().perform()

                video_id_find=chrome_driver.find_element(By.CSS_SELECTOR, html_dict["ad_id"])
                video_id=video_id_find.text.split("/")[0]
                ad_link=[f"https://www.youtube.com/watch?v={video_id}"]
                ad_number=len(ad_link)

            if ad_count_text=="Ad 1 of 2 ·":
                action.move_to_element(video_pause_find).click().perform()
                action.move_to_element(video_pause_find).context_click().perform()
                stats_find=chrome_driver.find_element(By.CSS_SELECTOR, html_dict["video_stats"])
                stats_find.click()

                video_id_find_1=chrome_driver.find_element(By.CSS_SELECTOR, html_dict["ad_id"])
                video_id_1=video_id_find_1.text.split("/")[0]
                ad_link_1=f"https://www.youtube.com/watch?v={video_id_1}"

                action.move_to_element(video_pause_find).click().perform()
                time.sleep(ad_duration_text)
                time.sleep(5)

                video_id_find_2=chrome_driver.find_element(By.CSS_SELECTOR, html_dict["ad_id"])
                video_id_2=video_id_find_2.text.split("/")[0]
                ad_link_2=f"https://www.youtube.com/watch?v={video_id_2}"

                ad_link=list(set([ad_link_1, ad_link_2]))
                ad_number=len(ad_link)

        except:
            ad_link="No ads found"
            ad_number=0
        
        printing_to_check=f"{target_video_name}: {ad_search_time}, {ad_link}"
        # print(printing_to_check)
        chrome_driver.refresh()

        time_list.append(ad_search_time)
        ad_list.append(ad_link)
        ad_number_list.append(ad_number)

    video_title_list=[target_video_name for i in range(number_of_fresh_times)]
    video_link_list=[target_video_link for i in range(number_of_fresh_times)]

    RESULT=pd.DataFrame({
        "HKT_watching":time_list,
        "video_name":video_title_list,
        "video_link": video_link_list,
        "ad_embedded":ad_list,
        "#_ads_found": ad_number_list
    })

    chrome_driver.quit()

    return RESULT

#################################################################################################################### 
# variables and arguments
#################################################################################################################### 
region="hk"
today_date=datetime.datetime.today().date()

search_terms_excel=r"C:\Users\Noel\Desktop\Alcohol_marketing\Python\DataCollection_SearchTerms.xlsx"
yt=pd.read_excel(search_terms_excel, sheet_name="youtube")
yt_link_list=yt["link"]

html_dict_defined={
    "video_pause":'#movie_player > div.ytp-chrome-bottom > div.ytp-chrome-controls > div.ytp-left-controls > button',
    "ad_count": 'ytp-ad-simple-ad-badge',
    "ad_duration": 'ytp-ad-duration-remaining',
    "video_stats": 'body > div.ytp-popup.ytp-contextmenu > div > div > div:nth-child(7) > div.ytp-menuitem-label',
    "ad_id": '#movie_player > div.html5-video-info-panel > div > div:nth-child(1) > span'
}

#################################################################################################################### 
# RUN 
#################################################################################################################### 
data=[]
for index, link in enumerate(yt_link_list):
    # print(index)
    scraper_data=youtube_ads_scraper(
        local_chrome_driver_path=r"C:\Users\Noel\Desktop\Alcohol_marketing\Python\chromedriver.exe",
        target_video_link=link, target_video_name=yt["Most popular YouTube videos based on total global views as of April 2022"][index],
        html_dict=html_dict_defined,
        number_of_fresh_times=2,
    ); data.append(scraper_data)
    time.sleep(5)

print("Done!")

output=pd.concat(data).reset_index(drop=True)
display(output); list(output["ad_embedded"])

C:\Users\Noel\AppData\Local\Temp\ipykernel_42044\3222538095.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrome_driver=webdriver.Chrome(local_chrome_driver_path)


Done!


,HKT_watching,video_name,video_link,ad_embedded,#_ads_found
0,13:29:22,Baby Shark Dance,https://www.youtube.com/watch?v=XqZsoesa55w,No ads found,0
1,13:29:24,Baby Shark Dance,https://www.youtube.com/watch?v=XqZsoesa55w,[https://www.youtube.com/watch?v=C_dWNRt0_b8 ],1
2,13:29:57,Despacito,https://www.youtube.com/watch?v=kJQP7kiw5Fk,No ads found,0
3,13:29:59,Despacito,https://www.youtube.com/watch?v=kJQP7kiw5Fk,"[https://www.youtube.com/watch?v=GW8Thytc48Q ,...",2
4,13:30:36,Johny Johny Yes Papa,https://www.youtube.com/watch?v=7GjOOyBoELw,No ads found,0
5,13:30:38,Johny Johny Yes Papa,https://www.youtube.com/watch?v=7GjOOyBoELw,"[https://www.youtube.com/watch?v=Q8tNMgdz-4A ,...",2
6,13:31:10,Shape of You,https://www.youtube.com/watch?v=JGwWNGJdvx8,No ads found,0
7,13:31:12,Shape of You,https://www.youtube.com/watch?v=JGwWNGJdvx8,"[https://www.youtube.com/watch?v=yO87HpGJlms ,...",2
8,13:31:46,See You Again,https://www.youtube.com/watch?v=RgKAFK5djSk,No ads found,0
9,13:31:49,See You Again,https://www.youtube.com/watch?v=RgKAFK5djSk,"[https://www.youtube.com/watch?v=2kYojmh6seU ,...",2


['No ads found',
 ['https://www.youtube.com/watch?v=C_dWNRt0_b8 '],
 'No ads found',
 ['https://www.youtube.com/watch?v=GW8Thytc48Q ',
  'https://www.youtube.com/watch?v=bc2p0HrzYZU '],
 'No ads found',
 ['https://www.youtube.com/watch?v=Q8tNMgdz-4A ',
  'https://www.youtube.com/watch?v=xtSi5KBVtlc '],
 'No ads found',
 ['https://www.youtube.com/watch?v=yO87HpGJlms ',
  'https://www.youtube.com/watch?v=K3C0ahTm__M '],
 'No ads found',
 ['https://www.youtube.com/watch?v=2kYojmh6seU ',
  'https://www.youtube.com/watch?v=0Ab8edxqYP8 '],
 'No ads found',
 ['https://www.youtube.com/watch?v=_sr3M-EYSEQ '],
 'No ads found',
 ['https://www.youtube.com/watch?v=fwhKhmfu-o4 '],
 'No ads found',
 'No ads found',
 ['https://www.youtube.com/watch?v=0Ab8edxqYP8 ',
  'https://www.youtube.com/watch?v=N1SMuICDDO4 '],
 'No ads found',
 'No ads found',
 ['https://www.youtube.com/watch?v=FFBXQriGT44 ',
  'https://www.youtube.com/watch?v=sfwSfkuLl6U ']]